In [97]:
from mountaintools import client as mt
from spikeforest import SFMdaRecordingExtractor, SFMdaSortingExtractor
import spikeforestwidgets as SFW
import vdomr as vd
import numpy as np
import pandas as pd
#vd.config_jupyter()

# IMPORTANT - since this notebook is checked in to the repo... do not include image outputs.
# clear outputs before committing.

# This is a work in progress

# An example command for James:
# > ./assemble_website_data.py --output_ids hybrid_janelia_irc,paired_kampff_irc --dest_key_path output.json
# then put the output.json in the path below.

In [98]:
# grab this URL from the website archive, or create your own
# all the paths can also point to local files
mt.configDownloadFrom(['spikeforest.public', 'spikeforest.kbucket'])
#analysis_all = mt.loadObject(path='key://pairio/spikeforest/spike-front-results.json')
analysis_irc = mt.loadObject(path='key://pairio/spikeforest/spike-front-results-irc.json')

In [93]:
def find_study_analysis_result(A, study_name):
    for x in A['StudyAnalysisResults']:   
        try:
            if x['studyName'] == study_name:
                return x
        except:
            print(x.keys())
            return None
        
def find_sorter(A, sorter_name):
    for x in A['Sorters']:   
        if x['name'] == sorter_name:
            return x        
        
def find_sorting_result4sorter(A, sorter_name):
    list_sr = [x for x in A['SortingResults'] if x['sorterName'] == sorter_name]
    return list_sr

        
def find_study_analysis_result4sorter(A, sorter):
    list_sar = []
    for sar in A['StudyAnalysisResults']:
        #sarW = find_study_analysis_result(A, sar['studyName'])
        for sr in sar['sortingResults']:
            if sr['sorterName'] == sorter:
                list_sar.append(sr)
    return list_sar

def compute_avg_accuracy(sar, sorting_result, snr_threshold):
    snrs = sar['trueSnrs']
    x = sorting_result['accuracies']
    x_to_use = [x[i] for i in range(len(x)) if x[i] is not None and snrs[i] is not None and snrs[i] >= snr_threshold]
    if x_to_use:
        return np.mean(x_to_use)
    return 0


def display_table(A, snr_threshold=8):
    sorters = A['Sorters']
    for sset in A['StudySets']:
        display(vd.h5(sset['name']))
        table_rows = []
        for study in sset['studies']:
            sar = find_study_analysis_result(A, study['name'])
            row = dict(
                study=study['name']
            )
            for sr in sar['sortingResults']:
                sorter_name = sr['sorterName']
                val = compute_avg_accuracy(sar, sr, snr_threshold=snr_threshold)
                row[sorter_name] = val
            table_rows.append(row)
        df = pd.DataFrame(table_rows)
        df = df[['study'] + [sorter['name'] for sorter in sorters]].round(2)
        columnsTitles=["study","IronClust", "KiloSort2"]
        df=df.reindex(columns=columnsTitles)
        display(df)
        
        
def merge_analysis(from_website, update):
    # James: here is how to superimpose an updated analysis on top of an existing one:
    #from_website = mt.loadObject(path='sha1://d0eb11774305a926e75ad232e4a6b4a54ffed4b2/analysis.json')
    #update = mt.loadObject(path='key://pairio/spikeforest/spike-front-results-irc.json')
    #update = mt.loadObject(path='key://pairio/spikeforest/test1.json')
    #update = mt.loadObject(path='sha1://60d72c800e62b86afac17182fcd8c27f1d53d2fd/analysis.json')
    sorter_names_in_update = [s['name'] for s in update['Sorters']]
    for sr in from_website['SortingResults']:
        if sr['sorterName'] not in sorter_names_in_update:
            update['SortingResults'].append(sr)
    for sar in update['StudyAnalysisResults']:
        sarW = find_study_analysis_result(from_website, sar['studyName'])
        for sr in sarW['sortingResults']:
            if sr['sorterName'] not in sorter_names_in_update:
                sar['sortingResults'].append(sr)
    for sorter in from_website['Sorters']:
        if sorter['name'] not in sorter_names_in_update:
            update['Sorters'].append(sorter)
    return update
        
def select_sorter(A, sorter='IronClust'):   
    B = dict()
    B['SortingResults'] = find_sorting_result4sorter(A, sorter)
    B['StudyAnalysisResults'] = find_study_analysis_result4sorter(A, sorter)
    B['Sorters'] = find_sorter(A, sorter)
    
    B['mode'] = A['mode']
    B['StudySets'] = A['StudySets']
    B['Algorithms'] = A['Algorithms']
    B['General'] = A['General']
    return B

In [96]:
analysis_merge = merge_analysis(analysis_all, analysis_irc)
display_table(analysis_merge)

,study,IronClust,KiloSort2
0,paired_boyden32c,0.53,0.73


,study,IronClust,KiloSort2
0,paired_crcns,0.65,0.68


,study,IronClust,KiloSort2
0,paired_mea64c,0.83,0.82


,study,IronClust,KiloSort2
0,paired_kampff,0.92,0.97


,study,IronClust,KiloSort2
0,paired_monotrode_boyden32c,0.16,0
1,paired_monotrode_kampff,0.12,0
2,paired_monotrode_crcns,0.16,0
3,paired_monotrode_mea64c,0.41,0


...

,study,IronClust,KiloSort2
0,manual_tetrode_600s,0.39,0.44
1,manual_tetrode_1200s,0.39,0.46
2,manual_tetrode_2400s,0.36,0.46


,study,IronClust,KiloSort2
0,synth_mearec_neuronexus_noise10_K10_C32,0.98,0.97
1,synth_mearec_neuronexus_noise10_K20_C32,0.98,0.97
2,synth_mearec_neuronexus_noise10_K40_C32,0.95,0.92
3,synth_mearec_neuronexus_noise20_K10_C32,1.00,1.00
4,synth_mearec_neuronexus_noise20_K20_C32,0.99,0.97
5,synth_mearec_neuronexus_noise20_K40_C32,0.99,0.96


,study,IronClust,KiloSort2
0,synth_mearec_tetrode_noise10_K10_C4,0.98,0.00
1,synth_mearec_tetrode_noise10_K20_C4,0.95,0.78
2,synth_mearec_tetrode_noise20_K10_C4,0.98,0.00
3,synth_mearec_tetrode_noise20_K20_C4,0.97,0.00


,study,IronClust,KiloSort2
0,mea_c30,0.92,0.98


,study,IronClust,KiloSort2
0,hybrid_drift_siprobe,0.88,0.89
1,hybrid_static_siprobe,0.96,0.97
2,hybrid_drift_tetrode,0.78,0.71
3,hybrid_static_tetrode,0.92,0.89


In [94]:
#analysis_merge = merge_analysis(analysis_all, analysis_irc)
#display_table(analysis_all_irc, 6)
analysis_all_irc = select_sorter(analysis_all, sorter='IronClust')
display_table(analysis_all_irc)

#print(analysis_all.keys())
#analysis_all_irc['Algorithms']

dict_keys(['sorterName', 'accuracies', 'precisions', 'numMatches', 'numFalsePositives', 'numFalseNegatives', 'recalls', 'cpuTimesSec'])


TypeError: 'NoneType' object is not subscriptable

In [ ]:
a=analysis_all['StudyAnalysisResults'][1]['studyName']
display(a)

In [53]:
display_table(analysis_all)

,study,IronClust,KiloSort2
0,paired_boyden32c,0.6,0.73


,study,IronClust,KiloSort2
0,paired_crcns,0.67,0.68


,study,IronClust,KiloSort2
0,paired_mea64c,0.87,0.82


,study,IronClust,KiloSort2
0,paired_kampff,0.8,0.97


,study,IronClust,KiloSort2
0,paired_monotrode_boyden32c,0.19,0
1,paired_monotrode_kampff,0.12,0
2,paired_monotrode_crcns,0.15,0
3,paired_monotrode_mea64c,0.00,0


...

,study,IronClust,KiloSort2
0,manual_tetrode_600s,0.40,0.44
1,manual_tetrode_1200s,0.40,0.46
2,manual_tetrode_2400s,0.46,0.46


,study,IronClust,KiloSort2
0,synth_mearec_neuronexus_noise10_K10_C32,0.98,0.97
1,synth_mearec_neuronexus_noise10_K20_C32,0.98,0.97
2,synth_mearec_neuronexus_noise10_K40_C32,0.95,0.92
3,synth_mearec_neuronexus_noise20_K10_C32,1.00,1.00
4,synth_mearec_neuronexus_noise20_K20_C32,0.99,0.97
5,synth_mearec_neuronexus_noise20_K40_C32,0.99,0.96


,study,IronClust,KiloSort2
0,synth_mearec_tetrode_noise10_K10_C4,0.98,0.00
1,synth_mearec_tetrode_noise10_K20_C4,0.95,0.78
2,synth_mearec_tetrode_noise20_K10_C4,0.98,0.00
3,synth_mearec_tetrode_noise20_K20_C4,0.97,0.00


,study,IronClust,KiloSort2
0,mea_c30,0.92,0.98


,study,IronClust,KiloSort2
0,hybrid_drift_siprobe,0.89,0.89
1,hybrid_static_siprobe,0.96,0.97
2,hybrid_drift_tetrode,0.80,0.71
3,hybrid_static_tetrode,0.92,0.89
